In [1]:
import (
    "fmt"
    "sync"
    "sync/atomic"
    "time"
)

# Pattern for running a fixed # of goroutines

Start a fixed number of goroutines, and pass each the channel
to receive on. Have each goroutine range over the channel
to receive items. Have the main thread block on a "done" signal.
After the main thread receives the done signal, close the channel
that the goroutines are listening on.

In [2]:
MaxConcurrent := 4
var NumRunning int32

func handle(recv <-chan int) {
    for i := range recv {
        fmt.Println("running ", atomic.AddInt32(&NumRunning, 1), "threads at time: ", time.Now().Format(time.ANSIC))
        time.Sleep(500 * time.Millisecond)
        atomic.AddInt32(&NumRunning, -1)
    }
}

func Serve(recv <-chan int, quit <-chan bool) {
    for i := 0; i < MaxConcurrent; i++ {
        go handle(recv)
    }
    <-quit
}

ch := make(chan int, 100)
done := make(chan bool)

go Serve(ch, done)

for i := 0; i < 20; i++ {
    ch <- i
}
close(ch)
time.Sleep(5 * time.Second)

done <- true

running  1 threads at time:  Wed May 11 19:23:55 2022
running  4 threads at time:  Wed May 11 19:23:55 2022
running  2 threads at time:  Wed May 11 19:23:55 2022
running  3 threads at time:  Wed May 11 19:23:55 2022
running  4 threads at time:  Wed May 11 19:23:56 2022
running  4 threads at time:  Wed May 11 19:23:56 2022
running  4 threads at time:  Wed May 11 19:23:56 2022
running  4 threads at time:  Wed May 11 19:23:56 2022
running  4 threads at time:  Wed May 11 19:23:56 2022
running  4 threads at time:  Wed May 11 19:23:56 2022
running  4 threads at time:  Wed May 11 19:23:56 2022
running  4 threads at time:  Wed May 11 19:23:56 2022
running  4 threads at time:  Wed May 11 19:23:57 2022
running  4 threads at time:  Wed May 11 19:23:57 2022
running  4 threads at time:  Wed May 11 19:23:57 2022
running  4 threads at time:  Wed May 11 19:23:57 2022
running  3 threads at time:  Wed May 11 19:23:57 2022
running  4 threads at time:  Wed May 11 19:23:57 2022
running  3 threads at time: 

# sync.Cond

`sync.Cond` is useful for broadcasting to multiple goroutines

In [3]:
var wg sync.WaitGroup
c := sync.NewCond(&sync.Mutex{})

ctr := 0

fmtNow := func() string {
    return time.Now().Format("15:04:05")
}

for i := 0; i < 4; i++ {
    wg.Add(1)
    go func(id int, c *sync.Cond) {
        defer wg.Done()
        c.L.Lock()
        for ctr < 5 {
            fmt.Println("goroutine: ", id, " has counter: ", ctr, " at: ", fmtNow())
            c.Wait()
        }
        c.L.Unlock()
        fmt.Println("goroutine: ", id, " unblocked at: ", fmtNow())
    }(i, c)
}

for ctr < 5 {
    time.Sleep(1 * time.Second)
    c.L.Lock()
    ctr += 1
    c.Broadcast()
    c.L.Unlock()
}

wg.Wait()

goroutine:  3  has counter:  0  at:  19:24:00
goroutine:  0  has counter:  0  at:  19:24:00
goroutine:  1  has counter:  0  at:  19:24:00
goroutine:  2  has counter:  0  at:  19:24:00
goroutine:  2  has counter:  1  at:  19:24:01
goroutine:  0  has counter:  1  at:  19:24:01
goroutine:  3  has counter:  1  at:  19:24:01
goroutine:  1  has counter:  1  at:  19:24:01
goroutine:  1  has counter:  2  at:  19:24:02
goroutine:  0  has counter:  2  at:  19:24:02
goroutine:  2  has counter:  2  at:  19:24:02
goroutine:  3  has counter:  2  at:  19:24:02
goroutine:  3  has counter:  3  at:  19:24:03
goroutine:  1  has counter:  3  at:  19:24:03
goroutine:  0  has counter:  3  at:  19:24:03
goroutine:  2  has counter:  3  at:  19:24:03
goroutine:  1  has counter:  4  at:  19:24:04
goroutine:  2  has counter:  4  at:  19:24:04
goroutine:  0  has counter:  4  at:  19:24:04
goroutine:  3  has counter:  4  at:  19:24:04
goroutine:  3  unblocked at:  19:24:05
goroutine:  2  unblocked at:  19:24:05
go